# Введение в MapReduce модель на Python


In [148]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [149]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [150]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [151]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [152]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [153]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [154]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [155]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [156]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [157]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [158]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [159]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [160]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [161]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [162]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.0829123031902945),
 (1, 2.0829123031902945),
 (2, 2.0829123031902945),
 (3, 2.0829123031902945),
 (4, 2.0829123031902945)]

## Inverted index

In [163]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [164]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [165]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [166]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [167]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.05981551187506884),
   (None, 0.07821110252114549),
   (None, 0.1027983532637673),
   (None, 0.10293941991646516),
   (None, 0.10451974150904364),
   (None, 0.20598026683601278),
   (None, 0.20864859284205461),
   (None, 0.2777263886102014),
   (None, 0.2780825872090301),
   (None, 0.38786179542043353),
   (None, 0.398675914878415),
   (None, 0.40079099966744147),
   (None, 0.42751593379591535),
   (None, 0.4342857309992194),
   (None, 0.4411837959513294)]),
 (1,
  [(None, 0.532696741195867),
   (None, 0.5540193651434818),
   (None, 0.6708298975319277),
   (None, 0.6752779863474464),
   (None, 0.6835299988358828),
   (None, 0.7140906764440873),
   (None, 0.7142882860269851),
   (None, 0.7211232358411006),
   (None, 0.7277334876695171),
   (None, 0.7940319381642306),
   (None, 0.7950725741240634),
   (None, 0.7966513383203353),
   (None, 0.8030262011529384),
   (None, 0.9033185772086588),
   (None, 0.9593872459440649)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [168]:
input_numbers = [1, 3, 12, 15, 8, 25, -30, 85, 300]

def RECORDREADER():
  for value in input_numbers:
    yield (0, value)

def MAP(id: int, value:int):
  yield(id, value)

def REDUCE(id: int, digits: list):
  yield (id, max(digits))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 300)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [169]:
# Пример входного списка чисел
input_numbers = [3, 5, 12, 2, 8]

def RECORDREADER():
  for value in input_numbers:
    yield (0, value)

def MAP(index: int, value:int):
  yield(index, value)

def REDUCE(id: int, digits: list):
  yield (id, sum(digits)/len(digits))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 6.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [170]:
input_collection = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

def RECORDREADER(key:str):
  for user in input_collection:
    yield (getattr(user, key), user)

def groupbykey(iterable):
  ixGroup = {}
  for key, value in sorted(iterable, key=lambda x:x[0]):
    if key in ixGroup:
      ixGroup[key].append(value)
    else:
      ixGroup[key] = [value]
  return ixGroup.items()

print(list(groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER("id"))))), '\n')
print(list(groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER("age"))))), '\n')
print(list(groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER("gender"))))), '\n')

[(0, [User(id=0, age=34, social_contacts=30, gender='female')]), (1, [User(id=1, age=21, social_contacts=120, gender='female')]), (2, [User(id=2, age=21, social_contacts=340, gender='female')]), (3, [User(id=3, age=54, social_contacts=760, gender='female')]), (4, [User(id=4, age=54, social_contacts=145, gender='male')])] 

[(21, [User(id=1, age=21, social_contacts=120, gender='female'), User(id=2, age=21, social_contacts=340, gender='female')]), (34, [User(id=0, age=34, social_contacts=30, gender='female')]), (54, [User(id=3, age=54, social_contacts=760, gender='female'), User(id=4, age=54, social_contacts=145, gender='male')])] 

[('female', [User(id=0, age=34, social_contacts=30, gender='female'), User(id=1, age=21, social_contacts=120, gender='female'), User(id=2, age=21, social_contacts=340, gender='female'), User(id=3, age=54, social_contacts=760, gender='female')]), ('male', [User(id=4, age=54, social_contacts=145, gender='male')])] 



### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [171]:
input_collection = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=0, age = 34, gender='female', social_contacts=30)
]

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for user in split:
        yield (user.id, user)

  split_size =  int(np.ceil(len(input_collection)/maps))
  for i in range(0, len(input_collection), split_size):
    yield RECORDREADER(input_collection[i:i+split_size])

def MAP(idUser:int, user: User):
  yield (idUser, user)

def REDUCE(idUser: int, avUser: list):
  yield (list(set(avUser)))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

5 key-value pairs were sent over a network.


[(0,
  [[User(id=0, age=34, social_contacts=30, gender='female')],
   [User(id=2, age=21, social_contacts=340, gender='female')]]),
 (1,
  [[User(id=1, age=21, social_contacts=120, gender='female')],
   [User(id=3, age=54, social_contacts=760, gender='female')]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [172]:
input_collection = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

def RECORDREADER():
  for user in input_collection:
    yield (user.id, user)

def MAP(id: int, user:User):
  if(user.gender == 'female'):
    yield(user, user)

def REDUCE(id: int, user: User):
  yield user

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[User(id=0, age=34, social_contacts=30, gender='female')],
 [User(id=1, age=21, social_contacts=120, gender='female')],
 [User(id=2, age=21, social_contacts=340, gender='female')],
 [User(id=3, age=54, social_contacts=760, gender='female')]]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [173]:
class Projection(NamedTuple):
  id: int
  age: int

input_collection = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

def RECORDREADER():
  for user in input_collection:
    yield (user.id, user)

def MAP(id: int, user:User):
  projection = Projection(user.id, user.age)
  yield(projection, projection)

def REDUCE(prj: Projection, avUser: list):
  yield (prj, prj)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Projection(id=0, age=34), Projection(id=0, age=34)),
 (Projection(id=1, age=21), Projection(id=1, age=21)),
 (Projection(id=2, age=21), Projection(id=2, age=21)),
 (Projection(id=3, age=54), Projection(id=3, age=54)),
 (Projection(id=4, age=54), Projection(id=4, age=54))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [174]:
input_collection1 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

input_collection2 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=5, age=47, gender='male', social_contacts=340),
    User(id=6, age=19, gender='female', social_contacts=24),
]

input_collection = [input_collection1, input_collection2]

def RECORDREADER():
  for user_list in input_collection:
    for user in user_list:
      yield (user.id, user)

def MAP(id: int, user:User):
  yield (user, user)

def REDUCE(user: User, avUser: list):
  yield (user, user)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[(User(id=0, age=34, social_contacts=30, gender='female'),
  User(id=0, age=34, social_contacts=30, gender='female')),
 (User(id=1, age=21, social_contacts=120, gender='female'),
  User(id=1, age=21, social_contacts=120, gender='female')),
 (User(id=2, age=21, social_contacts=340, gender='female'),
  User(id=2, age=21, social_contacts=340, gender='female')),
 (User(id=3, age=54, social_contacts=760, gender='female'),
  User(id=3, age=54, social_contacts=760, gender='female')),
 (User(id=4, age=54, social_contacts=145, gender='male'),
  User(id=4, age=54, social_contacts=145, gender='male')),
 (User(id=5, age=47, social_contacts=340, gender='male'),
  User(id=5, age=47, social_contacts=340, gender='male')),
 (User(id=6, age=19, social_contacts=24, gender='female'),
  User(id=6, age=19, social_contacts=24, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [175]:
input_collection1 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

input_collection2 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=5, age=47, gender='male', social_contacts=340),
    User(id=6, age=19, gender='female', social_contacts=24),
]

input_collection = [input_collection1, input_collection2]

def RECORDREADER():
  for user_list in input_collection:
    for user in user_list:
      yield (user.id, user)

def MAP(id: int, user:User):
  yield (user, user)

def REDUCE(user: User, avUser: list):
  if(len(avUser) == 2):
    yield (user, user)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[(User(id=0, age=34, social_contacts=30, gender='female'),
  User(id=0, age=34, social_contacts=30, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [176]:
input_collection1 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=1, age = 21, gender='female', social_contacts=120),
    User(id=2, age = 21, gender='female', social_contacts=340),
    User(id=3, age = 54, gender='female', social_contacts=760),
    User(id=4, age = 54, gender='male', social_contacts=145)
]

input_collection2 = [
    User(id=0, age = 34, gender='female', social_contacts=30),
    User(id=5, age = 21, gender='female', social_contacts=340),
    User(id=6, age = 19, gender='female', social_contacts=24),
]

input_collection = [input_collection1, input_collection2]

def RECORDREADER():
  for index, user_list in enumerate(input_collection):
    for user in user_list:
      yield (index, user)

def MAP(id: int, user:User):
  yield (user, id)

def REDUCE(user: User, avUser: list):
  if len(avUser) == 1 and avUser[0] == 0:
    yield (user, user)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[(User(id=1, age=21, social_contacts=120, gender='female'),
  User(id=1, age=21, social_contacts=120, gender='female')),
 (User(id=2, age=21, social_contacts=340, gender='female'),
  User(id=2, age=21, social_contacts=340, gender='female')),
 (User(id=3, age=54, social_contacts=760, gender='female'),
  User(id=3, age=54, social_contacts=760, gender='female')),
 (User(id=4, age=54, social_contacts=145, gender='male'),
  User(id=4, age=54, social_contacts=145, gender='male'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [177]:
class Product(NamedTuple):
    id: int
    category: str

class ProductCost(NamedTuple):
    category:str
    cost: int

input_collection1 = [
    Product(1, 'Ice cream'),
    Product(2, 'Banana')
]
input_collection2 = [
    ProductCost('Ice cream', 60),
    ProductCost('Banana', 20)
]

input_collection = [input_collection1, input_collection2]

def RECORDREADER():
  for index, user_list in enumerate(input_collection):
    for user in user_list:
      yield (index, user)

def MAP(id: int, obj:NamedTuple):
  if(id == 0):
     yield(obj.category, (0, obj.id))
  else:
     yield(obj.category, (1, obj.cost))

def REDUCE(name: str, lst: list):
  R = [obj for (id, obj) in lst if id == 0]
  S = [obj for (id, obj) in lst if id == 1]
  for a in R:
    for c in S:
      yield (a, name, c)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[(2, 'Banana', 20), (1, 'Ice cream', 60)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [178]:
class ProductCostByCategory(NamedTuple):
   category_id: int
   category: str
   cost: int

class Product(NamedTuple):
    id: int
    category: str

class ProductCost(NamedTuple):
    category:str
    cost: int

input_collection1 = [
    ProductCost('Ice cream', 65),
    ProductCost('Banana', 30),
    ProductCost('Ice cream', 85),
    ProductCost('Banana', 40)
]
input_collection2 = [
    ProductCost('Ice cream', 60),
    ProductCost('Banana', 20),
    ProductCost('Ice cream', 80),
    ProductCost('Banana', 25)
]

input_collection = [input_collection1, input_collection2]

def RECORDREADER():
  for index, user_list in enumerate(input_collection):
    for user in user_list:
      yield (index, user)

def MAP(id: int, productCostByCategory:ProductCostByCategory):
  yield (id, productCostByCategory.cost)

def REDUCE(id: int, lst: list):
  yield(id, sum(lst))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output


[(0, 220), (1, 185)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [179]:
from typing import Iterator
import numpy as np

matrix = np.ones((5, 5))
vector = np.array([i for i in range (5)])

maps = 3
reducers = 1

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        yield ((i, j), (split[i, j], vector[j], split.shape[1]))

  split_size =  int(np.ceil(len(matrix)/maps))
  for i in range(0, len(matrix), split_size):
    yield RECORDREADER(matrix[i:i+split_size])

def MAP(nums, values):
  i, j = nums
  mat_val, vec_val, cols = values
  yield ((i, cols), mat_val * vec_val)

def REDUCE(nums, res):
  i, cols = nums
  yield (i, sum(res) / (len(res) / cols))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

25 key-value pairs were sent over a network.


[(0, [(0, 10.0), (1, 10.0)])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [180]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [181]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for row in range(small_mat.shape[0]):
    yield ((row, k), v1 * small_mat[row][j])

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

Проверьте своё решение

In [182]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [183]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [184]:
def RECORDREADER():
  for i in range(big_mat.shape[1]):
    for j in range(small_mat.shape[0]):
      for k in range(small_mat.shape[1]):
        yield ((j, i), (0, small_mat[j, k]))

  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield ((i, k), (1, big_mat[j, k]))


def MAP(nums, values):
  yield (nums, values)


def REDUCE(nums, values):
  small_matrix_values = [value for index, value in values if index == 0]
  big_matrix_values = [value for index, value in values if index == 1]
  yield (nums, np.dot(small_matrix_values, big_matrix_values))


output = MapReduce(RECORDREADER, MAP, REDUCE)

np.allclose(reference_solution, asmatrix(output))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [185]:
def asmatrix_dist(reduce_output):
  reduce_output_list = []
  for key, values in reduce_output:
    for value in values:
      reduce_output_list.append(value)

  ixCount = {}
  for key, value in reduce_output_list:
    if key not in ixCount:
      ixCount[key] = 0
    ixCount[key] += value
  return asmatrix((key, value) for key, value in ixCount.items())

In [186]:
maps = 2
reducers = 1

def INPUTFORMAT():
  global maps
  small_matrix_rows = small_mat.shape[0]
  big_matrix_rows = big_mat.shape[0]


  def RECORDREADER_SMALL(split, idx):
    split_rows, split_cols = split.shape
    for i in range(big_mat.shape[1]):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((j + idx, i), (0, split[j, k]))


  split_size = int(np.ceil(small_matrix_rows / maps))
  for i in range(0, small_matrix_rows, split_size):
    yield RECORDREADER_SMALL(small_mat[i : i + split_size], i)


  def RECORDREADER_BIG(split):
    split_rows, split_cols = split.shape
    for i in range(small_matrix_rows):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((i, k), (1, split[j, k]))


  split_size = int(np.ceil(big_matrix_rows / maps))
  for i in range(0, big_matrix_rows, split_size):
    yield RECORDREADER_BIG(big_mat[i : i + split_size])


def MAP(nums, values):
  yield (nums, values)


def REDUCE(nums, values):
  small_matrix_values = [value for index, value in values if index == 0]
  big_matrix_values = [value for index, value in values if index == 1]
  yield (nums, np.dot(small_matrix_values, big_matrix_values))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]


np.allclose(reference_solution, asmatrix_dist(partitioned_output))

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [187]:
maps = 2
reducers = 1


def INPUTFORMAT():
  global maps
  small_rows = small_mat.shape[0]
  small_cols = small_mat.shape[1]

  big_rows = big_mat.shape[0]
  big_cols = big_mat.shape[1]

  def RECORDREADER_SMALL_1(split, idx):
    split_rows, split_cols = split.shape
    for i in range(big_mat.shape[1]):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((j + idx, i), (0, split[j, k]))

  def RECORDREADER_SMALL_2(split, idx):
    split_rows, split_cols = split.shape
    for i in range(big_mat.shape[1]):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((j + idx, i), (0, split[j, k]))


  def RECORDREADER_BIG_1(split, idx):
    split_rows, split_cols = split.shape
    for i in range(small_rows):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((i, k + idx), (1, split[j, k]))


  def RECORDREADER_BIG_2(split, idx):
    split_rows, split_cols = split.shape
    for i in range(small_rows):
      for j in range(split_rows):
        for k in range(split_cols):
          yield ((i, k + idx), (1, split[j, k]))


  small_readers = {
      0: RECORDREADER_SMALL_1,
      1: RECORDREADER_SMALL_2
  }


  big_readers = {
       0: RECORDREADER_BIG_1,
       1: RECORDREADER_BIG_2
  }

  split_size_rows = int(np.ceil(small_rows / maps))
  split_size_cols = int(np.ceil(small_cols / maps))

  counter = -1
  for i in range(0, small_rows, split_size_rows):
    for j in range(0, small_cols, split_size_cols):
      counter += 1
      yield small_readers[counter % len(small_readers)](small_mat[i : i + split_size_rows, j : j + split_size_cols], i)


  split_size_rows = int(np.ceil(big_rows / maps))
  split_size_cols = int(np.ceil(big_cols / maps))

  counter = -1
  for i in range(0, big_rows, split_size_rows):
    for j in range(0, big_cols, split_size_cols):
      counter += 1
      yield big_readers[counter % len(big_readers)](big_mat[i : i + split_size_rows, j : j + split_size_cols], j)


def MAP(nums, values):
  yield (nums, values)


def REDUCE(nums, values):
  small_matrix_values = [value for index, value in values if index == 0]
  big_matrix_values = [value for index, value in values if index == 1]
  yield (nums, np.dot(small_matrix_values, big_matrix_values))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

np.allclose(reference_solution, asmatrix_dist(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


True

Как мы можем заметить, данное решение будет работать только в том случае, когда мы передаем положение элемента